In [1]:
import pandas as pd

In [2]:
customer = pd.read_csv('uselog_joined.csv')
customer

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,end_date_c,duration
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14,NaN,CA1,0,オールタイム,10500,通常,8.000000,8.0,8,8,0,2019-04-30,1
4188,HD246549,XXXXX,C01,F,2019-03-14,NaN,CA1,0,オールタイム,10500,通常,10.000000,10.0,10,10,0,2019-04-30,1
4189,GD037007,XXXXX,C03,M,2019-03-14,NaN,CA1,0,ナイト,6000,通常,8.000000,8.0,8,8,0,2019-04-30,1
4190,OA953150,XXXXX,C01,M,2019-03-14,NaN,CA1,0,オールタイム,10500,通常,11.000000,11.0,11,11,0,2019-04-30,1


In [3]:
uselog_months = pd.read_csv('use_log_months.csv')
uselog_months

,ym,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7
...,...,...,...
36837,201903,TS995853,8
36838,201903,TS998593,8
36839,201903,TS999079,3
36840,201903,TS999231,6


In [4]:
# 顧客の退会を予測する

In [5]:
# 当月と前月の利用履歴のみを利用する
yms = list(uselog_months["ym"].unique())
uselog = pd.DataFrame()

for i in range(1, len(yms)):
    # 当月
    tmp = uselog_months.loc[uselog_months["ym"]==yms[i]]
    tmp.rename(columns={"count":"count_t"}, inplace=True)

    # 前月
    tmp_before = uselog_months.loc[uselog_months["ym"]==yms[i-1]]
    del tmp_before["ym"]
    tmp_before.rename(columns={"count":"count_p"}, inplace=True)
    
    # customer_id で結合
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    
    # ユニオン
    uselog = pd.concat([uselog, tmp], ignore_index=True)

C:\Users\likmh\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
uselog

,ym,customer_id,count_t,count_p
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0
...,...,...,...,...
33846,201903,TS995853,8,11.0
33847,201903,TS998593,8,7.0
33848,201903,TS999079,3,2.0
33849,201903,TS999231,6,6.0


In [7]:
# 退会顧客の退会前月のデータを作成する
# 退会月の前月に退会申請を出すシステムなので、退会月の前月のデータを見て、翌月に退会するかどうかを予測するのがねらい

In [21]:
from dateutil.relativedelta import relativedelta

In [22]:
exit_customer = customer.loc[customer["is_deleted"]==1] # 退会済
exit_customer["exit_date"] = None # 退会申請月（＝退会月の前月）
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)

C:\Users\likmh\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\likmh\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\likmh\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: h

In [23]:
exit_customer["ym"] = exit_customer["exit_date"].dt.strftime("%Y%m")

C:\Users\likmh\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
exit_customer

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,end_date_c,duration,exit_date,ym
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30 00:00:00,201803
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,デイタイム,7500,通常,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30 00:00:00,201803
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,1.0,1.0,1,1,0,2018-04-30,23,2018-03-30 00:00:00,201803
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,ナイト,6000,通常,3.5,3.5,4,3,1,2018-05-31,23,2018-04-30 00:00:00,201804
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-05-31,23,2018-04-30 00:00:00,201804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4049,OA143589,XXXXX,C03,F,2019-01-15,2019-02-28,CA1,1,ナイト,6000,通常,4.5,4.5,6,3,0,2019-02-28,1,2019-01-28 00:00:00,201901
4050,GD367382,XXXX,C01,M,2019-01-15,2019-03-31,CA1,1,オールタイム,10500,通常,7.0,6.0,12,3,0,2019-03-31,2,2019-02-28 00:00:00,201902
4068,OA034442,XXXXX,C02,M,2019-02-05,2019-03-31,CA1,1,デイタイム,7500,通常,7.0,7.0,11,3,0,2019-03-31,1,2019-02-28 00:00:00,201902
4086,PL508278,XXXXX,C03,M,2019-02-09,2019-03-31,CA1,1,ナイト,6000,通常,7.0,7.0,10,4,0,2019-03-31,1,2019-02-28 00:00:00,201902


In [25]:
uselog["ym"] = uselog["ym"].astype(str)

In [26]:
# 退会済の顧客の退会前月のデータと結合するので、それ以外の年月・顧客のデータは不要
# leftjoinして欠損値をdropnaしてもよいが、ここではinnerjoinで結合する
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "ym"], how="inner")

In [27]:
exit_uselog

,ym,customer_id,count_t,count_p,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,end_date_c,duration,exit_date
0,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500,通常,3.000000,3.0,3,3,0,2018-06-30,3,2018-05-30 00:00:00
1,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500,通常,3.000000,3.0,4,2,1,2018-06-30,7,2018-05-30 00:00:00
2,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500,通常,3.333333,3.0,5,2,0,2018-06-30,12,2018-05-30 00:00:00
3,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500,通常,3.333333,3.0,5,2,0,2018-06-30,4,2018-05-30 00:00:00
4,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000,通常,2.000000,1.0,4,1,0,2018-06-30,9,2018-05-30 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31,CA1,...,6000,通常,4.500000,4.5,7,1,0,2019-03-31,12,2019-02-28 00:00:00
1100,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31,CA3,...,6000,入会費無料,6.250000,6.0,8,5,0,2019-03-31,3,2019-02-28 00:00:00
1101,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31,CA3,...,7500,入会費無料,2.500000,2.0,5,1,0,2019-03-31,3,2019-02-28 00:00:00
1102,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31,CA1,...,7500,通常,4.250000,4.0,7,2,1,2019-03-31,12,2019-02-28 00:00:00


In [29]:
# 件数の確認 退会済顧客ごとに1件（退会前月のみ）のデータになっているはず
len(exit_uselog["customer_id"].unique())

1104